In [13]:
import json
from typing import List, Union
from pathlib import Path
import re

import pandas as pd
import requests
import yaml

In [14]:
CONFIG_FILENAME = "/home/jovyan/work/data/gtfs-22-nov/config_gtfs.yaml"#"config_gtfs.yaml"

In [15]:
def load_csv(filepath: Union[str, Path]) -> pd.DataFrame:
    """
    Carrega o arquivo CSV e retorna um DataFrame.
    """
    return pd.read_csv(filepath)

def dump_fixtures(fixtures: List[dict], filepath: Union[str, Path]):
    """
    Salva os fixtures em um arquivo JSON.
    """
    with open(filepath, "w") as f:
        json.dump(fixtures, f, indent=4)

def resolve_e(case,record,i):
    if case == 'trip_id[12:15]':
        return record['trip_id'][12:15]
    if case =='i':
        return str(i)
    if case == "": #TODO: resolver o caso da via.
        return ""
    if case == "stop_id[4:5]":
        return record["stop_id"][4:5]
    if case in ['3',3]:#set modal hard coded.
        return int(3)
    
#usado aqui para controlar a execução de fixtures sem alterar arquivos.
a={}
def gtfs_to_fixtures(dataframe, config_json, model, trip_list):
    fixtures = [] 
    k=config_json["json"]["fields"].keys()
    print(k)
    e=['trip_id[12:15]', "i","","stop_id[4:5]",3,"3"]
    for i, record in dataframe.iterrows():
        result = {
                "model": "pontos."+config_json["json"]["model"],
                "pk": record[config_json["json"]["pk"]] if config_json["json"]["pk"] not in e else resolve_e(config_json["json"]["pk"],record,i),
                "fields": dict.fromkeys(k,"")
            }
        
        #print(result)
        for item in k:
            if config_json["json"]["fields"][item] not in e:
                result["fields"][item]=record[config_json["json"]["fields"][item]]
            else:
                result["fields"][item]=resolve_e(config_json["json"]["fields"][item],record,i)
        
            if model =="agency":
                result["pk"]=str(result["pk"])
            if model =="sequence":
                result["pk"]=int(result["pk"])
            if model =="route":
                result["fields"]['agency']=str(result["fields"]['agency'])
            if model =="trip":
                #result["fields"]['direction']=int(result["fields"]['direction'])
                if result["fields"]['headsign']!=result["fields"]['headsign']:#check for NaN
                    result["fields"]['headsign']=""
                
            if model =="stop":
                result["fields"]['latitude']=str(result["fields"]['latitude'])
                result["fields"]['longitude']=str(result["fields"]['longitude'])
                #if result["fields"]['mode']=="": #erro gtfsv2 id sem mode, hardcoded
                #result["fields"]['mode']=3
            
            #if model in ["linha","route"]:
                #if result["fields"]['mode'] not in list(ID_MODAL.values()): #erro gtfsv2 id sem mode, hardcoded
                #result["fields"]['mode']=3

        #if model == "trip":
        #   if result["pk"] not in trip_list:
        #       trip_list.append(result["pk"])
        #       fixtures.append(result)
        if model == "sequence" and result["fields"]["trip"] in trip_list:# retira trips que são SNs
            fixtures.append(result)
        if model not in ["sequence"]:
            fixtures.append(result)
    
    dump_fixtures(fixtures, f"/home/jovyan/work/data/gtfs-22-nov/{model}.json")
    display(f"Adicionado {len(fixtures)} a {model}")

In [16]:
# casos especiais:
#(caso1) id_modal>30 and id_modal<40 --> I
#(caso2) not id_modal --> J
ID_MODAL={
    'MM':'M',
    '10':'M',
    '11':'T',
    '12':'V',
    '13':'S',
    '14':'P',
    '20':'B',
    '21':'H', 
    '22':'O',
    '23':'E',
    '30':'I',
    '31 a 39':'I',
    '----':'J',
    '40':'L',
    '41':'C',
    '50':'F',
    '60':'A'
}

def translate_to_new_mode(mode):
    
    if mode!='MM' and int(mode)>30 and int(mode)<39:
        return "I"
    if str(mode) not in ID_MODAL:
        return "J"
    else:
        return ID_MODAL[str(mode)]
    

ID_CORREDOR={
    '0':'0',
    '1':'O',
    '2':'M',
    '3':'C',
    '4':'L',
    '5':'B',
    '6':'Z',
    '7':'Y'
}

def translate_to_new_corredor(corredor):
    return ID_CORREDOR[corredor]


TIPO_PARADA={
    '0':'0',
    '1':'C',
    '2':'A',
    '4':'P',
    '5':'E',
    '6':'S',
    '7':'U',
    '8':'X',
    '9':'T'
}
def translate_to_new_parada(parada):
    return TIPO_PARADA[parada]

print('dicionarios de para geração dos ids preparados')

def treat_results(stop,code,count,i):
    return {
            "model": "pontos.qrcode",
            "pk": count+i,
            "fields": {
                "stop": stop,
                "code": code,
            }
    }

def convert_old_to_new_stop(stop):
    return stop[0:1]+stop[3:6]+translate_to_new_mode(stop[6:8])+translate_to_new_corredor(stop[8])+stop[-6:-2]+translate_to_new_parada(stop[-2])+stop[-1:]
    #return stop

dicionarios de para geração dos ids preparados


In [17]:
config = yaml.load(
        open(CONFIG_FILENAME, "r"), 
        Loader=yaml.FullLoader
    )

config["models"].keys()

dict_keys(['agency', 'linha', 'route', 'trip', 'stop', 'sequence', 'qrcode'])

In [18]:
trip_list=[]
#usado para gerar todos os arquivos de fixture descritos em config.yml 
#trip precisa ser gerando antes de sequence 
for model in config["models"].keys():
    config_json=config["models"][model]
    dataframe = pd.read_csv( config_json['source'] )
    if model == "sequence":
        #trip_list=pd.json_normalize(json.load(open("../fixtures/trip.json")))['pk'].tolist()
        trip_list=pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/trip.json")))['pk'].tolist()
        dataframe["trip_id"]= dataframe["trip_id"].str.split("_").str[0]
        dataframe.drop_duplicates(subset=['trip_id', 'stop_id'], keep="first", inplace=True)
        dataframe = dataframe.loc[dataframe["trip_id"].isin(trip_list)]
        
        #retirando stop_id inexistentes
        stop_list=pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/stop.json")))['pk'].tolist()
        dataframe = dataframe.loc[dataframe["stop_id"].isin(stop_list)==True]
        display(dataframe.loc[dataframe["stop_id"].isin(stop_list)==False])
     
    if model == "trip":
        trip_list=[]
        #route_list=pd.json_normalize(json.load(open("../fixtures/route.json")))['pk'].tolist()
        route_list=pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/route.json")))['pk'].tolist()
        dataframe=dataframe.loc[dataframe["route_id"].isin(route_list)]
        #dataframe=dataframe.loc[dataframe["trip_short_name"].str.contains("(SN)")==False]
        dataframe["trip_id"]= dataframe["trip_id"].str.split("_").str[0]
        dataframe.drop_duplicates(subset='trip_id', keep="first", inplace=True)
        
        ####correção em caso de NaNs no campo direction#####
        df_na = dataframe.loc[dataframe["direction_id"].isna()==True]
        dataframe=dataframe.loc[dataframe["direction_id"].isna()==False]
        
    if model == "qrcode2":
        dataframe=dataframe.loc[dataframe["stop_code"].isna()==False]
 

    if model in ["route","linha"]:
        dataframe=dataframe.loc[dataframe["route_short_name"].str.contains("(SN)")==False]
        
    display(f"{model}  {dataframe.shape[0]}")
    gtfs_to_fixtures(dataframe, config_json, model,trip_list)


'agency  5'

dict_keys(['name'])


'Adicionado 5 a agency'

/tmp/ipykernel_73/3146840706.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataframe=dataframe.loc[dataframe["route_short_name"].str.contains("(SN)")==False]


'linha  370'

...

dict_keys(['trip', 'stop', 'order'])


'Adicionado 38262 a sequence'

'qrcode  8997'

dict_keys(['stop', 'code'])


'Adicionado 8997 a qrcode'

In [7]:
# stop_id faltantes em sequence
df = pd.read_csv("/home/jovyan/work/data/gtfs-22-nov/stop_times.txt"  )
trip_list=pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/trip.json")))['pk'].tolist()
df["trip_id"]= df["trip_id"].str.split("_").str[0]
df.drop_duplicates(subset=['trip_id', 'stop_id'], keep="first", inplace=True)


stop_list=pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/stop.json")))['pk'].tolist()
print(df["stop_id"].loc[df["stop_id"].isin(stop_list)==False].unique())

/tmp/ipykernel_73/1181182058.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/jovyan/work/data/gtfs-22-nov/stop_times.txt"  )


['m3ta']


In [8]:
#coferencia de NaNs em direction para trips
#trip_id="O0837AAA0ACDU01"
#trip = pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/trip.json")))
#trip.loc[trip["pk"]==trip_id]
#trip.loc[trip["fields.direction"].isna()]

display(df_na)

,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
486,175b00bb-a3ea-468b-93a2-bf6c9ff8aced,O0825AAA0A,U,Jesuítas,825,NaN,NaN,qi1g,NaN,NaN


In [9]:
s =  pd.read_csv("/home/jovyan/work/data/gtfs-22-nov/stop_times.txt")
trip_list=pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/trip.json")))['pk'].tolist()
s["trip_id"]= s["trip_id"].str.split("_").str[0]
s.drop_duplicates(subset=['trip_id', 'stop_id'], keep="first", inplace=True)
s = s.loc[s["trip_id"].isin(trip_list)]

#retirando stop_id inexistentes
stop_list=pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/stop.json")))['pk'].tolist()
#s = s.loc[s["stop_id"].isin(stop_list)]
display(s.loc[s["stop_id"].isin(stop_list)==False])


/tmp/ipykernel_73/2883189033.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  s =  pd.read_csv("/home/jovyan/work/data/gtfs-22-nov/stop_times.txt")


,trip_id,stop_sequence,stop_id,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint
332811,O0833AAA0AVDU02,39,m3ta,00:00:00,00:00:00,NaN,0.0,0.0,1.0,1.0,12191.887294,NaN


In [213]:
# generate qr-code.

convert_old_to_new_stop("206028220002310")

'2028O00023C0'

x


0     1BJG
1     1EBQ
2     1G30
3     1K84
4     1LBP
5     1LVC
6     1T1W
7     1VYH
8     2DAX
9     2HSG
10    2L88
11    2L8R
12    2PVS
13    2TDL
14    31HK
15    3A7T
16    3FP6
17    3GDT
18    3GJF
19    3JH6
Name: code, dtype: object

In [233]:
import requests
last_page =True
#url='https://api.dev.mobilidade.rio/qrcode/'
url="https://api.mobilidade.rio/qrcode/"
fixtures=[]
i=0
while last_page:
    x = requests.get(url)
    x=json.loads(x.text)
    results=x['results']
    for item in results:
        code = item["code"]
        stop=item['stop']["id"]
        fixtures.append(treat_results(stop,code,0,i))
        i+=1

    if x["next"]:
        url= x["next"]
    else:
        last_page=False

print(len(fixtures),'pontos adicionados a fixtures')

51 pontos adicionados a fixtures


In [29]:
fixtures=[]

In [19]:
def treat_results(stop,code,count,i):
    return {
            "model": "pontos.qrcode",
            "pk": count+i,
            "fields": {
                "stop": stop,
                "code": code,
            }
    }


def dump_fixtures(fixtures: List[dict], filepath: Union[str, Path]):
    """
    Salva os fixtures em um arquivo JSON.
    """
    with open(filepath, "w") as f:
        json.dump(fixtures, f, indent=4)
        
        
fixtures=[]


new_code=[    
["1K84","206028220002310"],
["3GDT","206028220002710"],
["6EIC","206028220002910"],
["6PUF","206028220003110"],
["7DDY","206028220003210"],
["7GIN","206028220003410"],
["9VQU","206029220003810"],
["AWBW","206027220010510"],
["FMCV","206027220005110"],
["HKBW","206027220009810"],
["IGR3","206027220006210"],
["JJAY","206027220006110"],
["RE3Z","206027220006010"],
["VIN5","2027O00067C0"],
["YXB9","2021O00007C0"], #JB
    
["7M9B","1005O00198C0"],
["7KKY","1005O00197C0"], #passos
    
["JG2L","1005O00180C0"],
["JUCL","1005O00181C0"],
["CZ9H","t2u9"], # Carioca
    
["DQ2D","3061O00018C0"],
["T5TT","3061O00019C0"],
["HFGL","3061O00020C0"],
["DRVV","3061O00021C0"],
["TDBA","3061O00022C0"],
["DQ3X","3061O00023C0"], #bom retiro
    
["1BJG","2035O00028S9"],
["1EBQ","2035O00039S9"],
["1G30","2036O00012S9"],
["1LBP","2036O00050S9"],
["1LVC","2036O00045S9"],
["1T1W","2036O00013S9"],
["1VYH","2036O00051S9"],
["2DAX","2036O00052S9"],
["2HSG","2036O00014S9"],
["2L8R","2036O00015S9"],
["2L88","2036O00053S9"],
["2PVS","2036O00054S9"],
["2TDL","2036O00016S9"],
["Q3VX","2036O00055S0"], # 28
    
["3A7T","2036O00001S9"],
["EN4C","2036O00043S9"],
["FFD3","2036O00002S9"],
["FL96","2036O00003S9"],
["3FP6","2036O00004S9"],
["3GJF","2036O00056S9"],
["3JH6","2036O00057S9"],
["FU3N","2036O00044S9"],
["FYF6","2036O00058S9"],
["G56G","2036O00005S9"],
["4AZE","2036O00006S9"], # teodoro
    
["4HKB","2036O00007S0"],
["5GT3","2036O00049S0"],
["5SGU","2035O00043S0"],
["7V3P","2035O00018S0"],
["8A9A","2035O00019S0"],
["8AES","2035O00002S0"],
["8HAM","2035O00041S0"],
["8UHE","2033O00021S0"],
["9D2Q","2033O00022S0"],
["9E8I","2033O00245S0"],
["9NWD","2033O00244S2"],
["31HK","2033O00045S0"],
["47K3","2033O00046S0"]#São francisco Xavier
]

for i in range(len(new_code)):
        code = new_code[i][0]
        stop = new_code[i][1]
        #print(type(stop))
        #print(len(stop))
        if len(stop)==15:
            stop=convert_old_to_new_stop(stop)
            print(stop,code)

        fixtures.append(treat_results(stop,code,len(fixtures),0))
        
print(len(fixtures),'pontos adicionados a fixtures')


2028O00023C0 1K84
2028O00027C0 3GDT
2028O00029C0 6EIC
2028O00031C0 6PUF
2028O00032C0 7DDY
2028O00034C0 7GIN
2029O00038C0 9VQU
2027O00105C0 AWBW
2027O00051C0 FMCV
2027O00098C0 HKBW
2027O00062C0 IGR3
2027O00061C0 JJAY
2027O00060C0 RE3Z
64 pontos adicionados a fixtures


In [20]:
dump_fixtures(fixtures, f"/home/jovyan/work/data/gtfs-22-nov/qrcode.json")
#dump_fixtures(fixtures, f"qrcode3.json") 

In [21]:
# teste para unicidade da tupla (code,stop)
code = pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/qrcode.json")))
stop = pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/stop.json")))
#display (stop)
display(len(code["fields.stop"].unique().tolist()), len(code["fields.code"].unique().tolist()))


display("qrcode[stop_id] diferente em stop",code.loc[code["fields.stop"].isin(stop["pk"])==False])





main = pd.json_normalize(json.load(open(f"qrcode4.json")))
display("main",main.loc[main["fields.stop"].isin(code["fields.stop"])==False])


#testando para saber se na geração altomatica pelo gtfs e o manual estão em conformidade.

manual = pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/qrcode.json")))
gtfs = pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs-22-nov/qrcode2.json")))
display("pontos que existem em manual e em gtfs", manual["fields.code"].loc[manual["fields.code"].isin(gtfs["fields.code"])==True].count())

c = manual.loc[manual["fields.code"].isin(gtfs["fields.code"])==True]
display("codes que existem em manual e em gtfs", c["fields.code"].count())
display("pontos que existem em manual e em gtfs", c["fields.stop"].loc[c["fields.stop"].isin(gtfs["fields.stop"])==True].count())

#print(manual["fields.code"].tolist())

64

64

'qrcode[stop_id] diferente em stop'

,model,pk,fields.stop,fields.code


'main'

...

64

'codes que existem em manual e em gtfs'

64

'pontos que existem em manual e em gtfs'

64

In [269]:
#Teste a para associação com stops.
#usando gtfs-nov, anterior foi gtfsv2
#try code
c="4HKB".upper()
print(code.loc[code["fields.code"]==c])


Empty DataFrame
Columns: [model, pk, fields.stop, fields.code]
Index: []


In [190]:
qr_code = pd.json_normalize(json.load(open(f"qrcode3.json")))
stop = pd.json_normalize(json.load(open("/home/jovyan/work/data/gtfs_nov/stop.json")))
#display(stop)
#print (len(qr_code["fields.code"].unique().tolist()),len(qr_code["fields.stop"].unique().tolist()))
display("qrcode[stop_id] diferente em stop",qr_code.loc[qr_code["fields.stop"].isin(stop["pk"])==False])

qr_code1=qr_code.loc[qr_code["fields.stop"].isin(stop["pk"])==False]
qr_code2 = pd.json_normalize(json.load(open(f"qrcode4.json")))
N_main = qr_code2["model"].count()
qr_code2 = qr_code2.loc[qr_code2["fields.stop"].isin(stop["pk"])==True]

display(f"Dos {N_main} qrcodes em main, {qr_code2['model'].count()} tem seus stop_ids associados em stop")

display(f"Dos {qr_code['model'].count()} qrcodes, {qr_code1['model'].count()} não tem seus stop_ids associados em stop")


display(" qrcode[stop_id] em main que mudaram e nao estão na lista acima",qr_code1.loc[qr_code1["fields.code"].isin(qr_code2["fields.code"])==False])

'qrcode[stop_id] diferente em stop'

,model,pk,fields.stop,fields.code
0,pontos.qrcode,0,206028220002310,1K84
1,pontos.qrcode,1,206028220002710,3GDT
2,pontos.qrcode,2,206028220002910,6EIC
3,pontos.qrcode,3,206028220003110,6PUF
4,pontos.qrcode,4,206028220003210,7DDY
5,pontos.qrcode,5,206028220003410,7GIN
6,pontos.qrcode,6,206029220003810,9VQU
7,pontos.qrcode,7,206027220010510,AWBW
8,pontos.qrcode,8,206027220005110,FMCV
9,pontos.qrcode,9,206027220009810,HKBW


'Dos 51 qrcodes em main, 51 tem seus stop_ids associados em stop'

'Dos 64 qrcodes, 13 não tem seus stop_ids associados em stop'

' qrcode[stop_id] em main que mudaram e nao estão na lista acima'

,model,pk,fields.stop,fields.code


In [179]:
a= qr_code2.loc[qr_code2["fields.code"].isin(qr_code1["fields.code"])==True].reset_index(drop=True)

b= qr_code1.loc[qr_code1["fields.code"].isin(qr_code2["fields.code"])==True]

b["old_stops"] = a["fields.stop"]

#stop.loc[stop["stop_id"]]

display (b)


display(stop["pk"].loc[stop["pk"].isin(qr_code["fields.stop"])==True].count())

,model,pk,fields.stop,fields.code,old_stops
0,pontos.qrcode,0,206028220002310,1K84,2028O00023C0
1,pontos.qrcode,1,206028220002710,3GDT,2028O00027C0
2,pontos.qrcode,2,206028220002910,6EIC,2028O00029C0
3,pontos.qrcode,3,206028220003110,6PUF,2028O00031C0
4,pontos.qrcode,4,206028220003210,7DDY,2028O00032C0
5,pontos.qrcode,5,206028220003410,7GIN,2028O00034C0
6,pontos.qrcode,6,206029220003810,9VQU,2029O00038C0
7,pontos.qrcode,7,206027220010510,AWBW,2027O00105C0
8,pontos.qrcode,8,206027220005110,FMCV,2027O00051C0
9,pontos.qrcode,9,206027220009810,HKBW,2027O00098C0


51

In [ ]:
#generate mode.json from api

In [73]:
import requests

url='https://api.mobilidade.rio/mode/'
fixtures=[]
x = requests.get(url)
x=json.loads(x.text)
results=x['results']
for item in results:
    modal={
            "model": "pontos.mode",
            "pk": translate_to_new_mode(item["id"]),
            "fields": {
                "name": item["name"],
            }
                
    }
    fixtures.append(modal)

print(len(fixtures),'modais adicionados a fixtures')
dump_fixtures(fixtures, f"../fixtures/mode.json")

11 modais adicionados a fixtures
